In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
#File Path

file_path= '../input/credit-card-customers/BankChurners.csv'

data = pd.read_csv(file_path, index_col='CLIENTNUM')

data = data.drop('Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', axis=1)
data = data.drop('Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', axis=1)

data.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Obtain target and predictors
y= data['Attrition_Flag']   #target variable


X= data # X is same as data
X = X.drop('Attrition_Flag', axis=1) # remove target column from the data and creates predictors in X
#X= X.drop('CLIENTNUM', axis=1) #remove CLIENTNUM column from the data


#y.head()
#print(y)
#print("Below is X.head()")
#X.head()

# Dividing Data into training data (X_train, y_train) and validation data (X_valid, y_valid)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
#Dealing with Categorical Data

#Encoding the target variable y

from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_y_train = y_train.copy()
label_y_valid = y_valid.copy()

# Apply label encoder to y_train and y_valid
#### label_y_train/valid=1 if 'Existing Customer', else it is 0
label_encoder = LabelEncoder()
label_y_train = label_encoder.fit_transform(y_train) 
label_y_valid = label_encoder.transform(y_valid)

#j=0
#for i in y_train:
#    if i == "Existing Customer":
#        label_y_train[j]=0
#        print(label_y_train[j])
#        j=j+1
#    else:
#        label_y_train[j]=1
#        print(label_y_train[j])
#        j=j+1
    

#j=0
#for i in y_valid:
#    if j=="Existing Customer":
#        label_y_valid[j]=0
#        print(label_y_valid[j])
#        j=j+1
#    else:
#        label_y_valid[j]=1
#        print(label_y_valid[j])
#        j=j+1

# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

#One-Hot encoding
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

#y_train.head()
#print(y_train)
#print("\nHi")
#for i in label_y_train: 
#    print(i)
#print("\nafter label_y_train")
#for i in label_y_valid:
#    print(i)

    



In [ ]:
#Dealing with Missing Data

from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(OH_X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = OH_X_train.columns
imputed_X_valid.columns = OH_X_valid.columns

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Function for comparing different approaches
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(imputed_X_train, label_y_train)
preds = model.predict(imputed_X_valid)
print(preds)
print(label_y_valid)
accuracy= accuracy_score(label_y_valid, preds)*100
print("Accuracy is: ", accuracy, "%")
# Save test predictions to file
output = pd.DataFrame({'Id': imputed_X_valid.index,
                       'Attrition prediction': preds,
                       'actual_y_valid': label_y_valid})
output.to_csv('submission.csv', index=False)
print(output)